In [1]:
# Importacao das libraries
import pandas as pd
import functions as main
import numpy as np
from scipy import stats
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

# Load Data

In [5]:
csv = "df_lagged_weekly"

opencsv = pd.read_csv(rf"D:\OneDrive - Instituto Politecnico de Castelo Branco\002_Learning\001_Pos_Graduacao\Ciclo 5 - Projeto Final\ProjetoFinal-PosGraduacaoEngenhariaFinanceira\{csv}.csv", sep=",")
opencsv.set_index('Date', inplace=True)
opencsv.index = pd.to_datetime(opencsv.index)
opencsv = opencsv.asfreq(pd.infer_freq(opencsv.index))
opencsv = opencsv.astype(float)
opencsv = opencsv.dropna()

In [6]:
df_model = opencsv.copy()

In [7]:
df_model.isnull().sum()

L1_BTC_Supply                  0
L1_BalanceSheet_FED            0
L1_BalanceSheet_global         0
L1_Bitcoin_Wikipedia_Search    0
L1_Dollar_Index                0
L1_GLD                         0
L1_HashRate                    0
L1_M2_Liquidity                0
L1_Miners_Revenue_USD          0
L1_Mining_Difficulty           0
L1_Nr_Addresses                0
L1_Nr_Transactions             0
L1_OIL_USD                     0
L1_QQQ                         0
L1_TLT                         0
L1_Ten_YR_USA                  0
L1_USD_OnChain_Volume          0
L1_VIX                         0
BTC_USD                        0
dtype: int64

## Seleção de Variáveis

Considerar técnicas como regressão lasso, ridge ou elastic net, que fazem seleção de variáveis automaticamente, pode ser útil se você suspeitar que nem todas as 20 variáveis são relevantes.

<div class="alert alert-block alert-info"><b>Hipoteses a testar</b>

H1: Ceteris paribus, Bitcoin é um ativo de especulação.

H2: Ceteris paribus, a performance de Bitcoin é influenciada por fatores macroeconómicos.

H3: Ceteris paribus, quanto mais liquidez há no mercado, maior é a apreciação de preço de Bitcoin.

H4: Ceteris paribus, os fatores endógenos de Bitcoin são os maiores explicadores da performance de Bitcoin.
</div>

In [8]:
df_model.columns

Index(['L1_BTC_Supply', 'L1_BalanceSheet_FED', 'L1_BalanceSheet_global',
       'L1_Bitcoin_Wikipedia_Search', 'L1_Dollar_Index', 'L1_GLD',
       'L1_HashRate', 'L1_M2_Liquidity', 'L1_Miners_Revenue_USD',
       'L1_Mining_Difficulty', 'L1_Nr_Addresses', 'L1_Nr_Transactions',
       'L1_OIL_USD', 'L1_QQQ', 'L1_TLT', 'L1_Ten_YR_USA',
       'L1_USD_OnChain_Volume', 'L1_VIX', 'BTC_USD'],
      dtype='object')

## Regressao Linear Multipla

In [22]:
# Yt = b0 + b1X1t + b2X2t + εt


# Modelos endogenos Bitcoin
# 'L1.BTC_Supply', 'L1.Bitcoin_Wikipedia_Search', 'L1.HashRate', 'L1.Miners_Revenue_USD', 
# 'L1.Mining_Difficulty', 'L1.Nr_Addresses', 'L1.Nr_Transactions', 'L1.Nr_Tx_Block', 'L1.USD_OnChain_Volume'
model1a = [ 'L1_Bitcoin_Wikipedia_Search', 'L1_Miners_Revenue_USD', 'L1_Mining_Difficulty', 'L1_USD_OnChain_Volume'] # 'L1.Nr_Addresses', 'L1.Nr_Transactions'
model1b = [ 'L1_Nr_Transactions', 'L1_HashRate', 'L1_BTC_Supply', ] # 'L1.Nr_Addresses', 'L1.Nr_Transactions'


# Modelos Macro Economicos
# 'L1.10YR_USA', 'L1.BalanceSheet_FED', 'L1.BalanceSheet_global', 'L1.Dollar_Index', 
# 'L1.GLD', 'L1.M2_Liquidity', 'L1.OIL_USD', 'L1.QQQ', 'L1.TLT', 'L1.VIX'

model2a = [ ]
model2b = [ ]

# Modelos Gerais
model3a = [  ]
model3b = [  ]



X = df_model[model1b]
y = df_model["BTC_USD"]

X = sm.add_constant(X) # Add a constant term to the predictors

model = sm.OLS(y, X)
# robust regression -> model.fit( cov_type = "HC3" ) 

results = model.fit( cov_type = "HC3" ) # Fit the OLS model 

print(results.summary()) # Print the summary of the regression results

                            OLS Regression Results                            
Dep. Variable:                BTC_USD   R-squared:                       0.933
Model:                            OLS   Adj. R-squared:                  0.933
Method:                 Least Squares   F-statistic:                     3911.
Date:                Thu, 29 Aug 2024   Prob (F-statistic):               0.00
Time:                        23:01:51   Log-Likelihood:                -251.37
No. Observations:                 705   AIC:                             510.7
Df Residuals:                     701   BIC:                             529.0
Df Model:                           3                                         
Covariance Type:                  HC3                                         
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                -25.1116      5

## Step Wise regression

In [14]:
import statsmodels.formula.api as smf

def forward_selected(data, endog, exg):
    '''
    Linear model designed by forward selection based on p-values.
    Parameters:
    -----------
    data : Pandas DataFrame with dependent and independent variables
    endog: string literals, dependent variable from the data
    exg: string literals, independent variable from the data
    Returns:
    --------
    res : an "optimal" fitted Statsmodels linear model instance
    with an intercept selected by forward selection
    '''
    remaining = set(data.columns)
    remaining = [e for e in remaining if (e not in endog)&(e not in exg)]
    exg = [exg]

    scores_with_candidates = []
    for candidate in remaining:
        formula = '{} ~ {}'.format(endog,' + '.join(exg + [candidate]))
        score = smf.ols(formula, data).fit(cov_type = "HC3").pvalues[2]
        scores_with_candidates.append((score, candidate))
    scores_with_candidates.sort()

    for pval,candidate in scores_with_candidates:
        if pval < 0.1:
            exg.append(candidate)
    
    formula = '{} ~ {}'.format(endog, ' + '.join(exg))
    res = smf.ols(formula, data).fit()
    
    return res

In [15]:
res = forward_selected(data = df_model, endog = 'BTC_USD', exg = 'L1_Nr_Addresses')

In [16]:
print(res.model.formula)

BTC_USD ~ L1_Nr_Addresses + L1_Miners_Revenue_USD + L1_QQQ + L1_M2_Liquidity + L1_BalanceSheet_global + L1_USD_OnChain_Volume + L1_GLD + L1_BalanceSheet_FED + L1_HashRate + L1_Mining_Difficulty + L1_OIL_USD + L1_BTC_Supply + L1_Ten_YR_USA + L1_VIX + L1_Bitcoin_Wikipedia_Search + L1_Nr_Transactions


In [21]:
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                BTC_USD   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                     7896.
Date:                Thu, 29 Aug 2024   Prob (F-statistic):               0.00
Time:                        23:01:18   Log-Likelihood:                 634.36
No. Observations:                 705   AIC:                            -1235.
Df Residuals:                     688   BIC:                            -1157.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         

## Auto run Regressions

In [24]:
models_columns = ['model_1a', 'model_1b'] # , 'model_1c', 'model_2a', 'model_2b', 'model_2c', 'model_2d', 'model_3a', 'model_3b', 'model_3c'

def run_models_stats(df, models_list, y):
        count = 0
        for model_columns in models_list:
            X = df[model_columns]

            X = sm.add_constant(X)  # Add a constant term to the predictors

            model = sm.OLS(y, X)
            results = model.fit()  # Fit the OLS model
            
            print("\n####################################################################################################### \n\n")
            print("\nModel:", models_columns[count]) # columns name
            #print("Variable:", results.params.index) # set as index
            print(f"\nCoeff: {results.params}") # coefficients
            print("\nP-Value:", results.pvalues) # p-values
            print("\nR-Squared:", results.rsquared_adj)
            print("No. Obs.:", results.nobs)
            print("Cond. no.:", results.condition_number)
            count += 1
            print("\n####################################################################################################### \n\n")


def run_models(df, models_list, y):
    f = open(f"MLR_Results_Ciclo{ciclo}.txt", "w")
    count = 0
    for model_columns in models_list:
        X = df[model_columns]

        X = sm.add_constant(X)  # Add a constant term to the predictors

        model = sm.OLS(y, X)
        results = model.fit()  # Fit the OLS model
        
        f.write("\n####################################################################################################### \n\n")
        f.write(models_columns[count])
        f.write(" \n")
        f.write(f"{results.summary()}")
        count += 1
        f.write("\n####################################################################################################### \n\n")
    
    f.close()

In [25]:

models_list = [model1a, model1b]
y = opencsv["BTC_USD"]

for ciclo in range(1, 5):
    opencsv = pd.read_csv(rf"D:\OneDrive - Instituto Politecnico de Castelo Branco\002_Learning\001_Pos_Graduacao\Ciclo 5 - Projeto Final\ProjetoFinal-PosGraduacaoEngenhariaFinanceira\df_ciclo{ciclo}_weekly.csv", sep=",")

    
    run_models_stats(opencsv, models_list, y)

MissingDataError: exog contains inf or nans

In [ ]:
# list of all params 
dir(results)

# Testes

### Homocesdasticity

In [ ]:
import statsmodels.formula.api as smf

# https://www.geeksforgeeks.org/how-to-perform-a-breusch-pagan-test-in-python/

### Normality of Residuals

Assumption: The residuals should be approximately normally distributed.

Test:
Q-Q Plot: Compare the quantiles of residuals to a normal distribution. If residuals are normal, the points will lie approximately on the 45-degree line.

Shapiro-Wilk Test or Kolmogorov-Smirnov Test: Statistical tests for normality of residuals.

Transformation: If residuals are not normally distributed, consider transforming the dependent variable or using robust regression methods.

### Independence of Errors


Assumption: The residuals (errors) should be independent of each other.

Test:

Durbin-Watson Test: Detects the presence of autocorrelation in residuals, especially in time series data.

Plot Residuals vs. Time: For time series data, plot residuals against time to detect patterns or autocorrelation.

Transformation:
Add Lagged Variables: For time series, include lagged values of the dependent variable as predictors.

### No Omitted Variable Bias

Assumption: All relevant independent variables are included in the model.

Test:

Ramsey RESET Test: Checks for omitted variables by testing for non-linearity in the model.

Remedies:

Include Relevant Variables: Ensure all theoretically important variables are included in the model.
Use Model Selection Criteria: Such as AIC or BIC to guide variable selection.

## Teste de Multicolinearidade


Como você tem muitas variáveis, é importante verificar a multicolinearidade, especialmente porque as variáveis acumuladas podem estar altamente correlacionadas entre si. Testes como o VIF (Variance Inflation Factor) podem ajudar a identificar problemas.

Multicollinearity describes the state where the independent variables used in a study exhibit a strong relationship with each other. This can pose a problem in many cases as you would normally want your independent variables to be… independent.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices

#find design matrix for linear regression model using 'rating' as response variable 
y, X = dmatrices('BTC_USD ~ Nr_Transactions + L1.USD_OnChain_Volume + L1.BTC_Supply + L1.Mining_Difficulty', data=df_model, return_type='dataframe')

#calculate VIF for each explanatory variable
vif = pd.DataFrame()
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['variable'] = X.columns

#view VIF for each explanatory variable 
vif


#       Interpreting the VIF Values:
#       VIF < 5: Generally considered acceptable, indicating low to moderate multicollinearity.
#       VIF between 5 and 10: Indicates moderate to high multicollinearity, which may be problematic, but not necessarily requiring immediate removal.
#       VIF > 10: Strong multicollinearity, typically suggesting that the variable should be considered for removal or that further investigation is needed.